In [2]:
import datetime
now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

Current date and time : 
2022-11-07 23:40:06


In [ ]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install bert-for-tf2
!pip install sentencepiece
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

TF version:  2.10.0
Hub version:  0.12.0


In [ ]:
!pip install tf-hub-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
hub.__version__

'0.12.0'

## Import modules

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import bert
from tensorflow.keras.layers import Dropout, Dense
FullTokenizer = bert.bert_tokenization.FullTokenizer
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math

In [ ]:
max_seq_length = 128  # Your choice here.
# Load BERT and the preprocessing model from TF Hub.
preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
# Use BERT on a batch of raw text inputs.
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
input = preprocess(text_input)
sequence_output = encoder(input)["sequence_output"]
clf_output = sequence_output[:, 0, :]
clf_output = Dropout(0.1)(clf_output)
class1 = Dense(1024, activation='relu')(clf_output)
out = Dense(1, activation='sigmoid')(class1)

In [ ]:
preprocess_2 = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
encoder_2 = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
input = preprocess(text_input)
sequence_output = encoder(input)["sequence_output"]

In [ ]:
model_2 = Model(inputs=text_input, outputs=sequence_output)

In [ ]:
model = Model(inputs=text_input, outputs=out)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
with open('/content/drive/MyDrive/4222 Bert Dataset/chosen_verses',"rb") as fp:
  lyrics = pickle.load(fp)

In [ ]:
final_lyrics = []
for i in lyrics:
  cand_lyrics = ' '.join(i)
  final_lyrics.append(cand_lyrics[:max_seq_length])

In [ ]:
final_lyrics[42427] = "Oh boy. . Oh boy, the sunlight that shines on me Oh boy. Oh boy (It becomes a different world from yesterday). . Oh fate is a little lacking, this is a miracle. What I discovered that even I didn't really know. . In a fantasy drawn in a dream."

In [ ]:
final_lyrics[41533] = "You wiggle wiggle (wiggle wiggle wiggle). . (Hey, baby boy) Don't spin round and round, you. (Such a monster, such a monster). . (Hey baby boy) Don't go round and round"

In [ ]:
result_2 = model_2.predict(final_lyrics)

 728/1793 [===========>..................] - ETA: 4:19:02

In [ ]:
result = model.predict(final_lyrics)

In [ ]:
input = preprocess(final_lyrics)
non_dense_output = encoder(input)

In [ ]:
non_dense_output[0]

array([[0.67776126],
       [0.5871713 ],
       [0.6164219 ],
       ...,
       [0.65208614],
       [0.5605182 ],
       [0.6138694 ]], dtype=float32)

In [ ]:
from numpy import save
save('/content/drive/MyDrive/4222 Bert Dataset/bert_full_sequence_embedding.npy',non_dense_output)